# Graded Assignment 1

## Part 1: Implementation of the Eight Queens Puzzle n=8

In [132]:
import numpy as np
import random
import cProfile
from datetime import datetime

"""Implements one random board constellation in the eight queen puzzle"""
class EightQueensState:
    def __init__(self, state=None):
        self.n = 8
        self.state = np.random.randint(0, self.n, self.n)
        self.fitness_value = self.fitness()

    # Calculates the fitness of a board state
    def fitness(self):
        
        # Counts the number of queen attacks
        count = 0
        
        # Iterates through all rows and counts the attacks
        for i in range(self.n - 1):
            
            # Saves the queens position as well as the position of subsequent queens 
            right_queen_pos = np.array(self.state[i + 1:])
            queen_pos = np.array(self.state[i])
            
            # for each queen, add one for every queen to the right
            count += (queen_pos == right_queen_pos).sum()
            
            # for each queen, add one for every queen on the same diagonal
            diagonal = np.arange(1, self.n - i)
            upper_diagonal = queen_pos + diagonal
            lower_diagonal = queen_pos - diagonal
            
            count += (right_queen_pos == upper_diagonal).sum()
            count += (right_queen_pos == lower_diagonal).sum()
        
        # Returns fitness score, highest possible score is 28
        return 28 - count
    
    
    # Implements mutation by changing one queen position to any other position on the row
    def mutate(self, mut_rate):
        if np.random.rand() < mut_rate:
            
            # Determines which queen to mutate
            pos = np.random.randint(self.n)
            
            # Determines how to move the queen on the row
            value = np.random.randint(self.n)
            
            self.state[pos] = value
            self.fitness_value = self.fitness()

    # Determines, if goal state was reached
    def is_goal(self):
        return self.fitness() == 28

    # Prints out the current board state
    def print_current_state(self):
        print("  ", end='')
        for i in range(len(self.state)):
            print(f"{i} ", end='')
        print()
        for i in reversed(range(len(self.state))):
            print(f"{i}", end=' ')
            for j in self.state:
                if i == j:
                    print("X", end=' ')
                else:
                    print("O", end=' ')
            print()
            
        if self.is_goal():
            print(f"Goal state was reached! fitness: {self.fitness_value}")
        else:
            print(f"Goal state was not reached yet! fitness: {self.fitness_value}")
        print()

In [133]:
class GeneticAlgorithm:
    """Solves the 8 Queens problem by implementing a genetic algorithm"""
    def __init__(self, population=None, pop_size=10,
                 mut_rate=0.75, recomb_rate=1,
                 max_runs=2400):
        
        self.mut_rate = mut_rate
        self.recomb_rate = recomb_rate
        self.pop_size = pop_size
        self.max_runs = max_runs
        
        # Creates a list of length pop_size containing Eight Queens Puzzles  
        if population==None:
            self.population=[EightQueensState() for _ in range(pop_size)]
    
    # Implements recombination by iterating through the population
    # two adjustent parents are chosen to create child constellation
    def recomb(self):
        for i in range(len(self.population)-1):
            if i % 2 == 0:
                if np.random.rand() < self.recomb_rate:
                    # Determins index of recombination
                    rec = np.random.randint(1, self.population[i].n)
                    
                    # Copies array to list, performs recombination
                    state1 = self.population[i].state.tolist()
                    state2 = self.population[i+1].state.tolist()
                    state1[:rec], state2[:rec] = state2[:rec], state1[:rec]
                    
                    # Safes child arrays inplace
                    self.population[i].state = np.array(state1)
                    self.population[i+1].state = np.array(state2)
                    self.population[i].fitness_value = self.population[i].fitness()
                    self.population[i+1].fitness_value = self.population[i+1].fitness()

    # Implements the mutation part of the genetic algorithm
    def mutate(self):
        for state in self.population:
            state.mutate(self.mut_rate)
            
    # Performs natural selection on the population to create new population
    def create_new_pop(self):
        new_pop = []
        
        
        for j in range(self.pop_size//5):
            # Chooses a random subpopulation of 4
            sub_population = random.choices(self.population, k=4)
        
            # Determines the fitness of the subpopulation
            for i in range(5):
                chosen_state = sub_population[0]
                max_fitness_value = chosen_state.fitness_value
                for state in sub_population[1:]:
                    if state.fitness_value > chosen_state.fitness_value:
                        chosen_state = state
                        max_fitness_value = state.fitness_value
                
                # Picks state with highest fitness
                new_pop.append(chosen_state)
                
        if max([state.fitness_value for state in new_pop]) < 20:
            self.population = [EightQueensState() for _ in range(self.pop_size)]
        else:
            self.population = new_pop          
                
    # Runs all aspects of the genetic algorithm
    def run(self):
        best_performance = 0
        run_counter = 0
        while best_performance != 28 and run_counter != self.max_runs:
            
            # Performs natural selection on the population
            self.create_new_pop()
            
            # Performs 1 cicles of recombination on the population
            self.recomb()
            
            # Performs 1 mutation cicle on the population
            self.mutate()
            
            # Finds best performing child state
            best_index = np.argmax([state.fitness_value for state in self.population])
            best_performance = self.population[best_index].fitness_value
            run_counter += 1
        
        return best_performance, self.population[best_index], run_counter

In [23]:
# Instantiates the Eight Queens Puzzle
print("Here one Eight Queens Puzzle is instantiated, (X stands for the queens):")
print()
queen_puzzle = EightQueensState()
queen_puzzle.print_current_state()

print("Now one mutation round was performed on the state:")
print()
queen_puzzle.mutate(1)
queen_puzzle.print_current_state()

Here one Eight Queens Puzzle is instantiated, (X stands for the queens):

  0 1 2 3 4 5 6 7 
7 X O O O O O O O 
6 O O O X O O O O 
5 O O O O O X O O 
4 O O O O O O O O 
3 O O O O O O O O 
2 O O O O O O O O 
1 O O O O X O O X 
0 O X X O O O X O 
Goal state was not reached yet! fitness: 23

Now one mutation round was performed on the state:

  0 1 2 3 4 5 6 7 
7 X O O O O O O O 
6 O O O X O O O O 
5 O O O O O X O O 
4 O O O O O O O O 
3 O O O O O O O O 
2 O O O O O O O O 
1 O O O O X O O X 
0 O X X O O O X O 
Goal state was not reached yet! fitness: 23



In [24]:
print("Now two Eight queen puzzles are outputted:")
print()
my_alg = GeneticAlgorithm(pop_size=10,
                          max_runs=2400,
                          mut_rate=0.75,
                          recomb_rate=1)


for state in my_alg.population[:2]:
    state.print_current_state()

print("Now recombination is performed on both puzzles to yield child generations:")
print()
my_alg.recomb()
for state in my_alg.population:
    state.fitness_value = state.fitness()
    
for state in my_alg.population[:2]:
    state.print_current_state()

Now two Eight queen puzzles are outputted:

  0 1 2 3 4 5 6 7 
7 O X O O O X O O 
6 X O O O O O X O 
5 O O O O O O O X 
4 O O O O O O O O 
3 O O O X O O O O 
2 O O O O X O O O 
1 O O X O O O O O 
0 O O O O O O O O 
Goal state was not reached yet! fitness: 17

  0 1 2 3 4 5 6 7 
7 O O O O O O O X 
6 O O X O O O O O 
5 X O O O O O O O 
4 O O O X O O O O 
3 O O O O O O O O 
2 O O O O O O X O 
1 O X O O X O O O 
0 O O O O O X O O 
Goal state was not reached yet! fitness: 22

Now recombination is performed on both puzzles to yield child generations:

  0 1 2 3 4 5 6 7 
7 O O O O O X O O 
6 O O X O O O X O 
5 X O O O O O O X 
4 O O O O O O O O 
3 O O O X O O O O 
2 O O O O X O O O 
1 O X O O O O O O 
0 O O O O O O O O 
Goal state was not reached yet! fitness: 18

  0 1 2 3 4 5 6 7 
7 O X O O O O O X 
6 X O O O O O O O 
5 O O O O O O O O 
4 O O O X O O O O 
3 O O O O O O O O 
2 O O O O O O X O 
1 O O X O X O O O 
0 O O O O O X O O 
Goal state was not reached yet! fitness: 23



In [27]:
print("Finally, the genetic algorithm will find a solution state:")
print()

best_performance = 0
t1 = datetime.now()
while best_performance < 28:
        my_alg = GeneticAlgorithm(pop_size=5,
                                  max_runs=2400,
                                  mut_rate=0.2,
                                  recomb_rate=1)
        best_performance, state, run_counter = my_alg.run()
delta = datetime.now() - t1
state.print_current_state()
print(f"Time needed to arrive at solution: {delta.total_seconds()} seconds")
            

Finally, the genetic algorithm will find a solution state:

  0 1 2 3 4 5 6 7 
7 O O X O O O O O 
6 O O O O O X O O 
5 O O O O O O O X 
4 X O O O O O O O 
3 O O O O X O O O 
2 O O O O O O X O 
1 O X O O O O O O 
0 O O O X O O O O 
Goal state was reached! fitness: 28

Time needed to arrive at solution: 5.280134 seconds


In [136]:
print("Average time needed to find a solution:")
print()

timed_results = []
counter = 0
while counter < 5:
    t1 = datetime.now()
    best_performance = 0
    while best_performance != 28:
        my_alg = GeneticAlgorithm(pop_size=5,
                                  max_runs=2400,
                                  mut_rate=0.2,
                                  recomb_rate=1)
        best_performance, state, run_counter = my_alg.run()
    delta = datetime.now() - t1
    timed_results.append(delta.total_seconds())
    counter += 1
    print(counter, delta)
    
    
print("Average out of 5 trials: ", np.mean(timed_results))
print("Standard deviation:", np.std(timed_results))

Average time needed to find a solution:

1 0:00:08.228929
2 0:00:03.824707
3 0:00:05.388595
4 0:00:14.892236
5 0:00:10.615664
Average out of 5 trials:  8.5900262
Standard deviation: 3.9219606539095926


In [49]:
# Used to optimize parameters in above genetic algorithm
def param_opt():
    results = []
    for mutation_rate in [0.01, 0.05, 0.1, 0.2]:
        for recomb_rate in [0.25, 0.5, 0.75, 1]:
            for pop in [5, 10, 15, 20]:
                for max_runs in [1800, 2400, 3000]:
                    counter = 0
                    t1 = datetime.now()
                    t2 = datetime.now()
                    delta = t2 - t1
                    while delta.total_seconds() <= 30:
                        my_alg = GeneticAlgorithm(pop_size=pop,
                                                  max_runs=max_runs,
                                                  mut_rate=mutation_rate,
                                                  recomb_rate=recomb_rate)
                        best_performance, _, _ = my_alg.run()
                        delta = datetime.now() - t1
                        if best_performance == 28 and delta.total_seconds() <= 30:
                            counter += 1
                            print(counter)
                            results.append({f"{mutation_rate}, {recomb_rate}, {pop}, {max_runs}" : [delta, counter]})
                    print({f"{mutation_rate}, {recomb_rate}, {pop}, {max_runs}" : delta})
    return results
 
    
parameter_optimization = False
if parameter_optimization:
    results = param_opt()
    print(results)

## Part 2

In [111]:
"""Implements one random board constellation in the eight queen puzzle of any size n"""
class EightQueensStateModified:
    def __init__(self, n, state=None):
        self.n = n
        
        # The state is not completely randomly, but by propagating element (n-1) + 2
        predefined_state = []
        for n in range(self.n):
            # predefined_state.append(np.random.randint(self.n))
            if n == 0:
                predefined_state.append(np.random.randint(self.n))
            else:
                predefined_state.append((predefined_state[n-1]+2) % self.n)
        self.state = np.array(predefined_state)
        #self.state = np.random.randint(0, self.n, self.n)
        self.max_fitness = sum([i for i in range(self.n)])
        self.fitness_value = self.fitness()
    
    # Calculates the fitness of a board state
    def fitness(self):
        
        # Counts the number of queen attacks
        count = 0
        
        # Iterates through all rows and counts the attacks
        for i in range(self.n - 1):
            
            # Saves the queens position as well as the position of subsequent queens 
            right_queen_pos = np.array(self.state[i + 1:])
            queen_pos = np.array(self.state[i])
            
            # for each queen, add one for every queen to the right
            count += (queen_pos == right_queen_pos).sum()
            
            # for each queen, add one for every queen on the same diagonal
            diagonal = np.arange(1, self.n - i)
            upper_diagonal = queen_pos + diagonal
            lower_diagonal = queen_pos - diagonal
            
            count += (right_queen_pos == upper_diagonal).sum()
            count += (right_queen_pos == lower_diagonal).sum()
        
        # Returns fitness score, highest possible score is dependent on n
        return self.max_fitness - count
    
    
    # Implements mutation by changing one queen position to any other position on the row
    def mutate(self, mut_rate):
        if np.random.rand() < mut_rate:
           
            # Determines which queen to mutate
            pos = np.random.randint(self.n)
            
            # Determines how to move the queen on the row
            value = np.random.randint(self.n)
            
            self.state[pos] = value
            self.fitness_value = self.fitness()
            
            if self.fitness_value == self.max_fitness:
                return self.fitness_value, self

    # Determines, if goal state was reached
    def is_goal(self):
        return self.fitness() == self.max_fitness

    
    # Prints out the current board state
    def print_current_state(self):
        print("  ", end='')
        for i in range(len(self.state)):
            print(f"{i} ", end='')
        print()
        for i in reversed(range(len(self.state))):
            print(f"{i}", end=' ')
            for j in self.state:
                if i == j:
                    print("X", end=' ')
                else:
                    print("O", end=' ')
            print()
            
        if self.is_goal():
            print(f"Goal state was reached! fitness: {self.fitness_value}/{self.max_fitness}")
        else:
            print(f"Goal state was not reached yet! fitness: {self.fitness_value}/{self.max_fitness}")
        print()

In [112]:
queen_4_puzzle = EightQueensStateModified(n=4)
queen_4_puzzle.print_current_state()

  0 1 2 3 
3 O O O O 
2 O X O X 
1 O O O O 
0 X O X O 
Goal state was not reached yet! fitness: 4/6



In [113]:
class GeneticAlgorithmModified:
    """Solves the n Queens problem by implementing a genetic algorithm"""
    def __init__(self, n=8, 
                 population=None, pop_size=10,
                 mut_rate=0.75, recomb_rate=1,
                 max_runs=2400,
                 d=1):
        self.n = n
        self.mut_rate = mut_rate
        self.recomb_rate = recomb_rate
        self.pop_size = pop_size
        self.max_runs = max_runs
        self.temp = 1
        self.d = d
        
        # Creates a list of length pop_size containing Eight Queens Puzzles  
        if population==None:
            self.population=[EightQueensStateModified(self.n) for _ in range(pop_size)]
    
    # Implements recombination by iterating through the population
    # two adjustent parents are chosen to create child constellation
    def recomb(self):
        for i in range(len(self.population)-1):
            if i % 2 == 0:
                if np.random.rand() < self.recomb_rate * self.temp:
                    # Determins index of recombination
                    rec = np.random.randint(1, self.population[i].n)
                    
                    # Copies array to list, performs recombination
                    state1 = self.population[i].state.tolist()
                    state2 = self.population[i+1].state.tolist()
                    state1[:rec], state2[:rec] = state2[:rec], state1[:rec]
                    
                    # Safes child arrays inplace
                    self.population[i].state = np.array(state1)
                    self.population[i+1].state = np.array(state2)
                    self.population[i].fitness_value = self.population[i].fitness()
                    self.population[i+1].fitness_value = self.population[i+1].fitness()
                    
                    # Finish algorithm if goal state is reached
                    if self.population[i].fitness_value == self.population[i].max_fitness:
                        return self.population[i].max_fitness, self.population[i+1]
                    if self.population[i+1].fitness_value == self.population[i+1].max_fitness:
                        return self.population[i+1].max_fitness, self.population[i+1]

    # Implements the mutation part of the genetic algorithm
    def mutate(self):
        for state in self.population:
            state.mutate(self.mut_rate*self.temp)
            
    # Performs natural selection on the population to create new population
    def create_new_pop(self):
        new_pop = []
        
        
        for j in range(self.pop_size//5):
            # Chooses a random subpopulation of 4
            sub_population = random.choices(self.population, k=4)
        
            # Determines the fitness of the subpopulation
            for i in range(5):
                chosen_state = sub_population[0]
                max_fitness_value = chosen_state.fitness_value
                for state in sub_population[1:]:
                    if state.fitness_value > chosen_state.fitness_value:
                        chosen_state = state
                        max_fitness_value = state.fitness_value
                
                # Picks state with highest fitness
                new_pop.append(chosen_state)
                
        if max([state.fitness_value for state in new_pop]) < (chosen_state.max_fitness - self.n):
            self.population = [EightQueensStateModified(self.n) for _ in range(self.pop_size)]
        else:
            self.population = new_pop         
                
    # Runs all aspects of the genetic algorithm
    def run(self):
        best_performance = 0
        run_counter = 0
        temperature = 1 - (run_counter)
        while best_performance != 28 and run_counter != self.max_runs:
            
            # Performs natural selection on the population
            self.create_new_pop()
            
            # Performs 1 cicles of recombination on the population
            self.recomb()
            
            # Performs 1 mutation cicle on the population
            self.mutate()
            
            # Finds best performing child state
            best_index = np.argmax([state.fitness_value for state in self.population])
            best_performance = self.population[best_index].fitness_value
            run_counter += 1
            
            # Update temperature
            if run_counter > 1:
                self.temp = self.d / np.log(run_counter)
            
        return best_performance, self.population[best_index]

In [138]:
print("Prints out the n=4 puzzle:")
best_performance = 0
t1 = datetime.now()
while best_performance < sum([i for i in range(4)]):
        my_alg = GeneticAlgorithmModified(n=4,
                                          d=2,
                                          pop_size=5,
                                          max_runs=2400,
                                          mut_rate=1,
                                          recomb_rate=1)
        best_performance, state = my_alg.run()
delta = datetime.now() - t1
state.print_current_state()
print(f"Time needed to arrive at solution: {delta.total_seconds()} seconds")

Prints out the n=4 puzzle:
  0 1 2 3 
3 O X O O 
2 O O O X 
1 X O O O 
0 O O X O 
Goal state was reached! fitness: 6/6

Time needed to arrive at solution: 8.295269 seconds


In [139]:
print("Prints out the n=5 puzzle:")
best_performance = 0
t1 = datetime.now()
while best_performance < sum([i for i in range(5)]):
        my_alg = GeneticAlgorithmModified(n=5,
                                          d=2,
                                          pop_size=5,
                                          max_runs=2400,
                                          mut_rate=1,
                                          recomb_rate=1)
        best_performance, state = my_alg.run()
delta = datetime.now() - t1
state.print_current_state()
print(f"Time needed to arrive at solution: {delta.total_seconds()} seconds")

Prints out the n=5 puzzle:
  0 1 2 3 4 
4 X O O O O 
3 O O X O O 
2 O O O O X 
1 O X O O O 
0 O O O X O 
Goal state was reached! fitness: 10/10

Time needed to arrive at solution: 0.709126 seconds


In [140]:
print("Prints out the n=6 puzzle:")
best_performance = 0
t1 = datetime.now()
while best_performance < sum([i for i in range(6)]):
        my_alg = GeneticAlgorithmModified(n=6,
                                          d=2,
                                          pop_size=5,
                                          max_runs=2400,
                                          mut_rate=1,
                                          recomb_rate=1)
        best_performance, state = my_alg.run()
delta = datetime.now() - t1
state.print_current_state()
print(f"Time needed to arrive at solution: {delta.total_seconds()} seconds")

Prints out the n=6 puzzle:
  0 1 2 3 4 5 
5 O O X O O O 
4 O O O O O X 
3 O X O O O O 
2 O O O O X O 
1 X O O O O O 
0 O O O X O O 
Goal state was reached! fitness: 15/15

Time needed to arrive at solution: 1078.000291 seconds


In [141]:
print("Prints out the n=7 puzzle:")
best_performance = 0
t1 = datetime.now()
while best_performance < sum([i for i in range(7)]):
        my_alg = GeneticAlgorithmModified(n=7,
                                          d=2,
                                          pop_size=5,
                                          max_runs=2400,
                                          mut_rate=1,
                                          recomb_rate=1)
        best_performance, state = my_alg.run()
delta = datetime.now() - t1
state.print_current_state()
print(f"Time needed to arrive at solution: {delta.total_seconds()} seconds")

Prints out the n=7 puzzle:
  0 1 2 3 4 5 6 
6 O O O O O X O 
5 O X O O O O O 
4 O O O O X O O 
3 X O O O O O O 
2 O O O X O O O 
1 O O O O O O X 
0 O O X O O O O 
Goal state was reached! fitness: 21/21

Time needed to arrive at solution: 1.685408 seconds


In [129]:
print("Average time needed to find a solution:")
print()

timed_results = []
counter = 0
while counter < 5:
    t1 = datetime.now()
    best_performance = 0
    while best_performance != 28:
        my_alg = GeneticAlgorithmModified(pop_size=5,
                                          n=8,
                                          d=2,
                                          max_runs=2400,
                                          mut_rate=1,
                                          recomb_rate=1)
        best_performance, state = my_alg.run()
    state.print_current_state()
    delta = datetime.now() - t1
    timed_results.append(delta.total_seconds())
    counter += 1
    
    
print("Average out of 5 trials: ", np.mean(timed_results))
print("Standard deviation out of 5 trials", np.std(timed_results))

Average time needed to find a solution:

  0 1 2 3 4 5 6 7 
7 O O X O O O O O 
6 O O O O X O O O 
5 O X O O O O O O 
4 O O O O O O O X 
3 X O O O O O O O 
2 O O O O O O X O 
1 O O O X O O O O 
0 O O O O O X O O 
Goal state was reached! fitness: 28/28

  0 1 2 3 4 5 6 7 
7 O O X O O O O O 
6 O O O O X O O O 
5 O X O O O O O O 
4 O O O O O O O X 
3 X O O O O O O O 
2 O O O O O O X O 
1 O O O X O O O O 
0 O O O O O X O O 
Goal state was reached! fitness: 28/28

  0 1 2 3 4 5 6 7 
7 O O X O O O O O 
6 O O O O X O O O 
5 O X O O O O O O 
4 O O O O O O O X 
3 X O O O O O O O 
2 O O O O O O X O 
1 O O O X O O O O 
0 O O O O O X O O 
Goal state was reached! fitness: 28/28

  0 1 2 3 4 5 6 7 
7 O O X O O O O O 
6 O O O O X O O O 
5 O X O O O O O O 
4 O O O O O O O X 
3 X O O O O O O O 
2 O O O O O O X O 
1 O O O X O O O O 
0 O O O O O X O O 
Goal state was reached! fitness: 28/28

  0 1 2 3 4 5 6 7 
7 O O X O O O O O 
6 O O O O X O O O 
5 O X O O O O O O 
4 O O O O O O O X 
3 X O O O O O O O 
2

In [121]:
solution_dictionary = {
    4 : 2,
    5 : 10,
    6 : 4,
    7 : 40,
    8 : 92,
    9 : 352,
    10 : 724
}

In [143]:
# Finds all solutions

solutions_found = {}
for board_size in []:
    t1 = datetime.now()
    solutions = []
    while len(solutions) != solution_dictionary[board_size]:
        while best_performance != sum([i for i in range(board_size)]):
            my_alg = GeneticAlgorithmModified(n=board_size,
                                              d=2,
                                              pop_size=5,
                                              max_runs=2400,
                                              mut_rate=1,
                                              recomb_rate=1)
            best_performance, best_state = my_alg.run()
        if best_state.state not in solutions:
            solutions.append(best_state.state)
    solutions_found[board_size] = solutions
    delta = datetime.now() - t1
    print(f"All solutions for n: {board_size} found! Time needed: {delta.total_seconds()}")